In [14]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
from tqdm import trange

In [2]:
def get_song_urls():
    '''Extracts all the urls of the top 500 songs from rolling stone magazine and returns them as a list.'''
    page = requests.get('https://www.cs.ubc.ca/~davet/music/list/Best9.html')
    html = bs(page.content, 'html.parser')
    urls = []
    for element in html.find('table', class_ = 'music').find_all('a'):
        content_str = element.get_attribute_list('href')[0]
        if content_str ==  None:
            continue
        elif '../track/' in content_str:
            urls.append('https://www.cs.ubc.ca/~davet/music' + content_str[2:])
    return urls

In [199]:
def get_song_data(url):
    '''Scrapes the data of a given URL. Return a dictionary containing the artist, title, year, genre_list and lyrics'''
    page = requests.get(url)
    html = bs(page.content, 'html.parser')
    
    lyrics = ''
    artist, title = html.find('div', class_ = 'pagetitle').decode_contents().split('<br/>')

    for line_index in range(60):
        line_content = html.find_all('td')[line_index].find('a')
        if line_content != None:
            line_content = line_content.decode_contents()

            if line_content == 'Year':
                year = int(html.find_all('td')[line_index+1].find('a').decode_contents())
                #print(year)
            elif line_content == 'LyricWiki':
                lyrics = html.find_all('td')[line_index+1].decode_contents()
                #print(lyrics)

    return {'Artist':artist, 
            'Title':title, 
            'Year':year,  
            'Lyrics':lyrics}

In [212]:
#work with index on web pages as well in dataframe 
def main():
    columns = ['Artist', 'Title', 'Year', 'Lyrics']
    data = pd.DataFrame(columns = columns)
    urls = get_song_urls()
    for i in trange(len(urls)):

        song_data = get_song_data(urls[i])
        data.at[i] = song_data
        if song_data['Lyrics'] == '':
            print(f'No lyrics found at index {i}')
    data.to_csv('../data/raw/Rolling_Stone_Top_500.csv')

In [213]:
if __name__=='__main__':
    main()

 10%|█         | 50/500 [02:51<09:22,  1.25s/it] 

No lyrics found at index 49


 14%|█▍        | 72/500 [03:34<08:32,  1.20s/it]

No lyrics found at index 71


 57%|█████▋    | 284/500 [08:32<04:40,  1.30s/it]

...

 86%|████████▌ | 428/500 [12:31<01:35,  1.32s/it]

No lyrics found at index 427


 97%|█████████▋| 484/500 [13:54<00:46,  2.93s/it]

No lyrics found at index 483


100%|██████████| 500/500 [14:16<00:00,  1.71s/it]
